# Variance Analysis in Multistep Bootstrapping

In order to understand the $Q(\sigma)$ algorithm better, we will analyze and compare its performance to other multistep bootstrapping methods. Since $Q(\sigma)$ is tied to the off policy n-step tree backup algorithm, we thus compare it to the off-policy variants of n-step SARSA, n-step expected SARSA, and to the n-step tree backup algorithm.

## Bootstrapping multistep methods
As [1] notes, bootstrapping methods work best when applied over multiple time steps, such that a significant and recognizable state change can be observed. From [1] also, note that multistep methods are usually associated with eligibility traces, but in this benchmark we consider only the multistep component of these algorithms.

## Theory
### Off-policy N-step SARSA and N-step expected SARSA [1]
The regular SARSA(0) algorithm learns the value of state-action pairs and, after each transition from a nonterminal state $S_t$ applies the update equation:
$$ Q(S,A) \leftarrow Q(S,A) + \alpha [ R + \gamma Q(S', A') - Q(S,A)]$$

In off-policy n-step SARSA, we still learn the value of state-action pairs but only update our estimate after $n$ steps, weighing the update using the importance sampling ratio $\rho_{t}^{t+n}$ defined as the relative probability under the two policies of taking the n actions from $A_t$ to $A_{t+n-1}$:
$$\rho_t^{t+n} = \prod_{k=t}^{min(t+n-1, T-1)}  \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)}$$

And we use the update equation:

$$ Q_{t+n}(S_t,A_t) = Q_{t+n-1}(S_t,A_t) + \alpha \rho_{t+1}^{t+n} [ G_t^{(n)} - Q_{t+n-1}(S_t,A_t)] $$

with $G_t^{(n)}$ the n-step return defined in terms of estimated action values as:  

$$ G_t^{(n)} = R_{t+1} + \gamma R_{t+2} \cdots + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n},a) , n \geq 1, 0\leq t \leq T-n$$


For off-policy n-step expected SARSA, we simply change the computation of the n-step return to:
$$ G_t^{(n)} = R_{t+1} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n \sum_a \pi(a|S_{t+n}) Q_{t+n-1}(S_{t+n},a) , n \geq 1, 0\leq t \leq T-n$$ 

### N-step Tree Backup Algorithm [1]
In the n-step tree backup algorithm, we consider all possible actions from each state and estimate their values at step $t$. However we do not boostrap for the action that is actually taken at time $t$ as we therefore have a sample for this action. We thus go on to consider the state and actions that follow from having taken that action at time $t$, until $n$ steps have been exhausted.

Note that each of the action values of the possible actions at a specific state gets weighted by its probability of being taken, with the taken action weighing the whole tree underneath it (= all the possible actions originating from having taken that action). Therefore all the actions underneath a taken action $a$ at step $t$ will have their probabilities weighted by the probability of taking $a$ at step $t$.

We thus obtain the n-step return used in tree-backup:
$$G_t^{(n)} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{min(t+n-1, T-1)}\delta_k \prod_{i=t+1}^{k} \gamma \pi(A_i | S_i)$$

where $$\delta_t = R_{t+1} + \gamma V_{t+1} - Q_{t-1}(S_t, A_t)$$

This return is then used in the action-value update equation (same as n-step SARSA, without importance sampling): 
$$ Q_{t+n}(S_t,A_t) = Q_{t+n-1}(S_t,A_t) + \alpha [ G_t^{(n)} - Q_{t+n-1}(S_t,A_t)] $$

### N-Step $Q(\sigma)$ algorithm[1]
In the n-step $Q(\sigma)$ algorithm, we alternate between the ideas from Sarsa and the tree backup algorithm. Thus at each step, based on the value of $\sigma \in [0,1]$, the $Q(\sigma)$ update takes a fraction of the sample-based update (Sarsa) and the rest from the non sampling update (tree backup).
[1] thus generalizes the TD error $\delta_t$ to be a function of $\sigma$:
$$\delta_t = R_{t+1} + \gamma[\sigma_{t+1}Q_{t}(S_{t+1}, A_{t+1}) + (1-\sigma_{t+1}) V_{t+1}] - Q_{t-1}(S_t, A_t)$$

which is then used in the n-step return equation :
$$G_t^{(n)} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{min(t+n-1, T-1)}\delta_k \prod_{i=t+1}^{k} \gamma [(1-\sigma_i)\pi(A_i | S_i) + \sigma_i]$$

Finally we update the importance sampling ratio to take $\sigma$ into account:
$$\rho_t^{t+n} = \prod_{k=t}^{min(t+n-1, T-1)} (\sigma_k \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)} + 1 - \sigma_k)$$

We can then use the above equations in the off-policy update equation from off-policy Sarsa.


# Variance Brief Theoretical Background

## Off-Policy vs. On-Policy

We begin with analyzing the variance in off-policy methods vs. on-policy methods. From [1], we know that for off-policy learning, importance sampling "enables off-policy learning, but at the cost of increasing the variance of the updates." While weighted importance sampling [2] or adapative importance sampling (based on previous timestep variance)[3,6] can bring down this variance, typical n-step off-policy SARSA does not use this variance reduction technique as described in [1].

TODO: can we get some equations up in here to prove that importance sampling off-policy is indeed more variant than on-policy SARSA"

### Getting Rid of Importance Sampling (n-step Tree Backup)

In [4], where n-step Tree Backup is first introduced (without importance sampling in Section 4 of [4]), the authors bypass importance sampling by using backups from possible actions at each step combined with the probability of the target policy taking this action. This removes importance sampling and theoretically the variance introduced by it. Intuitively, this is the same bias-variance tradeoff as in Expected Sarsa. By using the expectation at each timestep we are bounding the variance (see [5] for more details on bias-variance tradeoff of Sarsa vs. Expected SARSA). 

### Combining Importance Sampling with Expecation in Tree Backup (i.e. n-step $Q(\sigma)$)

By adding function which balances sampling with the expectation backups of n-step tree backup, we can balance the variance. One can think of an easy extension such that the $\sigma$ provided to the algorithm is a function of the previous step's variance (similar to adaptive importance sampling or autostep [3,6]). However, say that we simply choose the $\sigma$ value to be a uniformly distributed random variable. TODO: maths here???

## Effect of n on variance??

TODO:

## Partial Sketch of Variance Analyses

Note: We think our analysis here is outside the scope of the assignment, but we include some of our partial derivations regardless for those that might be interested in completing them with us.

As in [5], we formulate the variance of n-step Sarsa as follows. Take $v_t = \rho \left[ G_t^{(n)} - Q_{t+n-1}(S_t, A_t) \right]$

We can then expand the variance term as follows:

$$Var(s,a) = E \left\{ (v_t)^2 \right\} - (E \left\{ v_t \right\} )^2$$

Ignoring the second term for now:

$$Var(s,a) = E \left\{ \rho^2 G_t^{(n)} + Q_{t+n-1}^2(s_t, a_t) - 2 \rho G_t^{(n)} Q_{t+n-1}(s_t, a_t) \right\}$$

Here, $G_t^{(n)} = \sum_{k=1}^n \gamma^{k-1} R_{t+k} + \gamma^n \sum_a \pi(a | s_{t+n}) Q_{t+n-1}(s_t, a)$ 

Similarly, for n-step tree backup: $v_t = \left[ \hat{G}_t^{(n)} - Q_{t+n-1}(S_t, A_t) \right]$, where $$\hat{G}_t^{(n)} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{min(t+n-1, T-1)}\delta_k \prod_{i=t+1}^{k} \gamma \pi(A_i | S_i)$$
and $$\delta_t = R_{t+1} + \gamma V_{t+1} - Q_{t-1}(S_t, A_t)$$ and $V_{t+1} = \sum_{a} \pi(a|S_{t+1}) Q(S_{t+1}, a)$

Thus, for n-strep tree backup the variance becomes partially (ignoring the $(E \left\{ v_t \right\} )^2$ term for now:

$$Var(s,a) = E \left\{\hat{G}_t^{(n)} + Q_{t+n-1}^2(s_t, a_t) - 2 \hat{G}_t^{(n)} Q_{t+n-1}(s_t, a_t) \right \}$$

Due to time we don't continue the derivations, but partially, we know that a main difference between the two terms is the importance sampling. We suggest that by expansion (reflecting what emperical evidence suggests in [2,4]), we expect that Sarsa will have higher variance when the $\rho$ term is larger (i.e. when importance sampling plays a larger role in the terms the difference in variance between tree backup, where there is no importance sampling, and n-step sarsa, where there is, will be larger).




# Practical Lessons

## Computational Efficiency (n-step methods aren't great)

Overall, we found n-step methods to be far more computationally intensive non-n-step variations on Sarsa. Some statistics on runtimes for 1000 episodes on machine with Intel i5 processor (6600):

+ N-step Sarsa (average runtime): 3.7 min
+ N-Step Expected Sarsa (average runtime): 5.15 min
+ N-Step Tree backup (average runtime): 5.35 min
+ N-Step Q-Sigma (average runtime): 8.3 min
+ Sarsa (average runtime): 29 seconds
+ Expected Sarsa (average runtime): 34 seconds
+ Double Expected Sarsa (average runtime): 35 seconds
+ Double Sarsa (average runtime): 32 seconds


## Citations and Footnotes
[1] Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. Vol. 1. No. 1. Cambridge: MIT press, 1998.

[2] Precup, Doina, Richard S. Sutton, and Sanjoy Dasgupta. "Off-policy temporal-difference learning with function approximation." ICML. 2001.

[3] Hachiya, Hirotaka, et al. "Adaptive importance sampling for value function approximation in off-policy reinforcement learning." Neural Networks 22.10 (2009): 1399-1410.

[4] Precup, Doina. "Eligibility traces for off-policy policy evaluation." Computer Science Department Faculty Publication Series (2000): 80.

[5] Van Seijen, Harm, et al. "A theoretical and empirical analysis of Expected Sarsa." Adaptive Dynamic Programming and Reinforcement Learning, 2009. ADPRL'09. IEEE Symposium on. IEEE, 2009.

[6] Mahmood, Ashique Rupam, et al. "Tuning-free step-size adaptation." Acoustics, Speech and Signal Processing (ICASSP), 2012 IEEE International Conference on. IEEE, 2012.
